In [1]:
import pandas as pd
train_file = "../data/train_data.txt"
test_file = "../data/test_data.txt"
answer_file = "../data/answer.txt"

In [2]:
%%time
df_train = pd.read_table(train_file, header=None, encoding='gb2312', sep=',')
df_test = pd.read_table(test_file, header=None, encoding='gb2312', sep=',')
df_answer = pd.read_table(answer_file, header=None, encoding='gb2312', sep=',')
df_train.rename(columns={1000:'label'}, inplace=True)
df_label = df_train['label']
df_label.shape
del df_train['label']
df_train.shape
print(df_train.shape)
print(df_label.shape)
print(df_test.shape)
print(df_answer.shape)

(8000, 1000)
(8000,)
(2000, 1000)
(20000, 1)
CPU times: user 2.57 s, sys: 271 ms, total: 2.84 s
Wall time: 4.9 s


In [15]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.239,0.079,0.934,0.131,0.024,0.748,0.951,0.922,0.858,0.475,...,0.202,0.747,0.097,0.141,0.314,0.268,0.851,0.746,0.554,0.951
1,0.972,0.852,0.330,0.114,0.003,0.383,0.192,0.180,0.860,0.654,...,0.571,0.333,0.052,0.201,0.495,0.631,0.017,0.925,0.047,0.327
2,0.118,0.368,0.989,0.331,0.944,0.539,0.118,0.867,0.351,0.956,...,0.119,0.122,0.170,0.270,0.917,0.503,0.068,0.160,0.751,0.784
3,0.044,0.476,0.882,0.202,0.445,0.290,0.162,0.620,0.428,0.610,...,0.606,0.178,-0.265,0.686,0.876,0.287,0.504,0.863,0.997,0.541
4,0.199,0.242,0.322,0.081,0.695,0.206,0.052,0.732,0.268,0.820,...,0.985,0.661,0.084,0.006,0.145,0.846,0.344,0.346,0.506,0.624


In [23]:
%%time
from sklearn.linear_model import LogisticRegression

def train(p):
    lr = LogisticRegression()
    lr.fit(df_train[p], df_label)

    predictY = lr.predict(df_test[p])
    ac, wa = 0, 0
    for py, y in zip(predictY, df_answer[0]):
        if py == y:
            ac += 1
        else:
            wa += 1
    rate = ac / (ac + wa)
    return rate

maxid, maxx = 0, 0
for i in range(1000):
    for j in range(i+1, 1000):
        p = [i, j]
        rate = train(p)
        if rate > maxx:
            maxid, maxx = i, rate
print(maxid, maxx)

In [13]:
%%time
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = svm.SVC(C=1.0, kernel='rbf', gamma=0.1)
clf.fit(df_train, df_label)

CPU times: user 1min 47s, sys: 2.17 s, total: 1min 49s
Wall time: 2min 10s


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
predictY = clf.predict(df_test)
ac, wa = 0, 0
for py, y in zip(predictY, df_answer[0]):
    if py == y:
        ac += 1
    else:
        wa += 1
print("正确: ", ac)
print("错误: ", wa)
print("准确率: ", ac / (ac + wa))

正确:  1374
错误:  626
准确率:  0.687
